# KeyPoints Extraction

In [ ]:
!pip install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 41.7 MB/s eta 0:00:00


## Loading the dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tr1gg3rtrash/yoga-posture-dataset")

print("Path to dataset files:", path)

100%|██████████| 447M/447M [00:16<00:00, 28.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1


## Importing Libraries

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
import os


## Initialising the model


In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils

## Function to extraxt and normalise keypoints

In [ ]:
def extract_keypoints(image_path):
    # Read the input image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Could not read image: {image_path}")
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image with MediaPipe Pose
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Extract keypoints
        keypoints = [(landmark.x, landmark.y, landmark.z) for landmark in results.pose_landmarks.landmark]
        keypoints = np.array(keypoints)

        # Normalize keypoints
        x_coords, y_coords = keypoints[:, 0], keypoints[:, 1]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        width = x_max - x_min
        height = y_max - y_min
        normalized_keypoints = (keypoints - [x_min, y_min, 0]) / [width, height, 1]

        return normalized_keypoints.flatten()
    else:
        print(f"No pose detected in image: {image_path}")
        return None

## Processing the entire dataset and save keypoints to CSV

In [ ]:
def process_dataset(dataset_path, output_csv):
    all_keypoints = []
    labels = []

    # Iterate through folders directly in the root directory
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):  # Ensure it is a folder
            print(f"Processing folder: {folder_path}")
            for file in os.listdir(folder_path):
                if file.lower().endswith((".jpg", ".jpeg", ".png")):  # Handle supported image formats
                    image_path = os.path.join(folder_path, file)
                    keypoints = extract_keypoints(image_path)
                    if keypoints is not None:
                        all_keypoints.append(keypoints)
                        labels.append(folder)  # Folder name as label

    if len(all_keypoints) > 0:
        # Save keypoints and labels to CSV
        df = pd.DataFrame(all_keypoints)
        df['label'] = labels
        df.to_csv(output_csv, index=False)
        print(f"Keypoints saved to {output_csv}")
    else:
        print("No keypoints were extracted. Please check the dataset and paths.")


In [ ]:
dataset_path = os.path.join(path)  # Base dataset path from kagglehub
output_csv = "/content/yoga_keypoints.csv"

# Process the dataset
process_dataset(dataset_path, output_csv)

Processing folder: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Anjaneyasana
Processing folder: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Virabhadrasana Three
No pose detected in image: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Virabhadrasana Three/File39.png
Processing folder: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Urdhva Mukha Svsnssana
No pose detected in image: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Urdhva Mukha Svsnssana/File38.png
No pose detected in image: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Urdhva Mukha Svsnssana/File20.png
Processing folder: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Ardha Chandrasana
No pose detected in image: /root/.cache/kagglehub/datasets/tr1gg3rtrash/yoga-posture-dataset/versions/1/Ardha Chandrasana/File43.

## Checking the CSV file

In [ ]:
import pandas as pd

keypoints_df = pd.read_csv('/content/yoga_keypoints.csv')
print(keypoints_df.head())

          0         1         2         3         4         5         6  \
0  0.379307  0.268986 -0.249039  0.403164  0.258503 -0.292665  0.405915   
1  0.542991  0.000873 -0.056974  0.560612  0.000000 -0.086612  0.561551   
2  0.582899  0.296683 -0.056250  0.609204  0.301483 -0.080810  0.610430   
3  0.364177  0.314397 -0.133954  0.384789  0.293991 -0.167073  0.389159   
4  0.497106  0.075594 -0.259878  0.524652  0.087406 -0.293495  0.527161   

          7         8         9  ...        90        91        92   93  \
0  0.260801 -0.292517  0.408872  ...  0.139663  0.943306  0.643350  1.0   
1  0.003118 -0.086581  0.562749  ...  0.990231  0.891121  0.301815  0.0   
2  0.308129 -0.080818  0.611666  ...  0.870198  0.907586 -0.086578  0.0   
3  0.294353 -0.166945  0.393661  ...  0.139931  0.972488  0.429078  1.0   
4  0.093818 -0.293323  0.530002  ...  0.125889  0.982878  0.225679  1.0   

         94        95   96        97        98         label  
0  1.000000 -0.332617  0.0  0.94101